In [ ]:
from typing import Tuple

%run imputers/interpolate.ipynb
%run imputers/median.ipynb
%run imputers/mode.ipynb
%run imputers/mean.ipynb
%run imputers/KNN_K=1.ipynb
%run imputers/KNN_K=5.ipynb
%run imputers/KNN_K=10.ipynb
%run imputers/KNN_K=15.ipynb
%run imputers/KNN_K=20.ipynb
%run imputers/KNN_K=100.ipynb
%run imputers/missforest.ipynb
%run imputers/hotdeck.ipynb
%run imputers/regression.ipynb
%run imputers/softimpute.ipynb
%run imputers/fillna.ipynb
%run imputers/mice.ipynb
%run imputers/time_interpolation.ipynb
%run imputers/linear_interpolation.ipynb
%run imputers/missForest_regressor_MICE.ipynb
%run imputers/bayesian_ridge_MICE.ipynb
%run imputers/forward_fill.ipynb
%run imputers/RNN.ipynb


imputers = {
    "median imputation": {
        "title": "Median imputation",
        "function": median_imputation
    },
    "mode imputation": {
        "title": "Mode imputation",
        "function": mode_imputation
    },
    "mean imputation": {
        "title": "Mean imputation",
        "function": mean_imputation
    },
    "fillna": {
        "title": "fillna",
        "function": fillna
    },
    "linear_interpolation": {
        "title": "linear_interpolation",
        "function": linear_interpolation
    },
    "time_interpolation" : {
        "title": "time_interpolation",
        "function" : time_interpolation
    },
    "missForest_regressor_MICE" : {
        "title": "missForest_regressor_MICE",
        "function" : ExtraTreesRegressor_MICE
    },
    "bayesian_ridge_MICE" : {
        "title": "bayesian_ridge_MICE",
        "function" : bayesian_ridge_MICE
    },
    "forward_fill" : {
        "title": "forward_fill",
        "function" : forward_fill
    },
    "Hot deck": {
        "title": "Hot deck",
        "prehook": hotdeck_prehook,
        "function": hotdeck
    },
    "Hot deck (classification)": {
        "title": "Hot deck",
        "prehook": hotdeck_prehook,
        "function": hotdeck_classification
    },
    "KNN k=1": {
        "title": "KNN k=1",
        "function": KNN_1
    },
    "KNN k=5": {
        "title": "KNN k=5",
        "function": KNN_5
    },
    "KNN k=10": {
        "title": "KNN k=10",
        "function": KNN_10
    },
    "KNN k=15": {
        "title": "KNN k=15",
        "function": KNN_15
    },
    "KNN k=20": {
        "title": "KNN k=20",
        "function": KNN_20
    },
    "KNN k=100": {
        "title": "KNN k=100",
        "function": KNN_100
    },
    "stochastic regression": {
        "title": "Stochastic regression",
        "function": lambda df, _: stochastic_regression(df, dfloader.targets)
    },
    "deterministic regression": {
        "title": "Deterministic regression",
        "function": lambda df, _: deterministic_regression(df, dfloader.targets)
    },
    "MICE": {
        "title": "MICE",
        "function": mice
    },
    "interpolation": {
        "title": "Interpolation",
        "function": interpolate
    },
    "MissForest imputation": {
        "title": "MissForest imputation",
        "function": MissForestImputation
    },
    "Softimpute": {
        "title": "Softimpute",
        "function": SoftImputation
    },
    "RNN": {
        "title": "RNN",
        "prehook": rnn_prehook,
        "function": rnn
    }
}

imputer_select = widgets.Dropdown(
    options=list(imputers.keys()),
    value=None,
    description='Imputer:'
)
display(imputer_select)
imputer = None
imputation_date = None
imputed_dfs: [pd.DataFrame] = None
imputer_configs = []

imputation_status = widgets.HTML(value="")
display(imputation_status)


def do_imputation():
    global imputed_dfs, imputer, imputation_date
    imputation_date = datetime.now().replace(second=0, microsecond=0)
    imputed_dfs = []
    for i in range(len(dfs_with_gaps)):
        imputation_status.value = f"Running imputation... ({i}/{len(dfs_with_gaps)})"
        dataset_config['current_gap_indices'] = gaps_indices[i]
        result, config = imputer["function"](dfs_with_gaps[i].copy(), dataset_config)
        imputer_configs.append(config)
        imputed_dfs.append(result)
    imputation_status.value = "Imputation complete"


def on_imputer_select_change(evt):
    global imputer
    imputer = imputers[evt.new]
    if "prehook" in imputer.keys():
        imputer["prehook"](lambda: do_imputation())
    else:
        do_imputation()


imputer_select.observe(on_imputer_select_change, 'value')
